In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft
from skimage import feature
import scipy
import numpy as np
import pandas as pd
from scipy.stats import linregress
from numba import njit, prange

In [2]:
df = pd.read_csv('../kaggle feat_eng/train_logs_corrected.csv')

In [15]:
ids = [
    "3a266257",
    "4b40b5c0",
    "08eb331a",
    "52bec922",
    "2717fdef",
    "f413c831",
    "29b7f2f6",
    "219e5338",
    "fd565688",
    "bf1a04b1",
    "3e10785d",
    "01963e20",
    "31d5a61d",
    "11612565",
    "fb6c9c09",
    "67d04f17",
    "e6693dd9"
]

In [17]:
scores = pd.read_csv('../data/train_scores.csv')

In [19]:
scores[scores['id'].isin(ids)]

,id,score
20,01963e20,4.5
86,08eb331a,2.5
164,11612565,2.0
326,219e5338,3.5
378,2717fdef,1.0
394,29b7f2f6,2.0
473,31d5a61d,1.5
545,3a266257,1.0
579,3e10785d,1.5
708,4b40b5c0,3.0


In [29]:
1139687/(60*1000)

18.994783333333334

In [30]:
df=df['idle_time']/(60*1000)

In [35]:
df.hist(3)

ValueError: Grouper for '3' not 1-dimensional

In [28]:
df[df['id']=='52bec922'].head(5)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,idle_time
2637428,52bec922,122,0,105,105,Input,q,q,q,1,229,1139687
2637429,52bec922,123,118,186,68,Input,q,q,q,2,229,1139687
2637430,52bec922,124,2403,2544,141,Input,q,q,q,3,229,1139687
2637431,52bec922,125,2600,2902,302,Input,q,q,q,4,229,1139687
2637432,52bec922,126,2747,2922,175,Input,q,q,q,5,229,1139687


In [23]:
df[df['id']=='ccbde4a9'].head(20)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
6615084,ccbde4a9,1,659,764,105,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615085,ccbde4a9,2,14628,14709,81,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615086,ccbde4a9,3,17248,17372,124,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615087,ccbde4a9,4,20598,20704,106,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615088,ccbde4a9,5,28519,28602,83,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615089,ccbde4a9,6,42135,42196,61,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615090,ccbde4a9,7,128414,128525,111,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615091,ccbde4a9,8,683447,683530,83,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615092,ccbde4a9,9,684332,684427,95,Nonproduction,Leftclick,Leftclick,NoChange,0,0
6615093,ccbde4a9,10,700519,700604,85,Nonproduction,Leftclick,Leftclick,NoChange,0,0


In [24]:
def process_group(group):
    # Find the first row where 'activity' equals 'Input' and 'text_change' equals 'q'
    first_input_q_index = group[(group['activity'] == 'Input') & (group['text_change'] == 'q')].index[0]

    # Calculate the 'down_time' of the first 'Input'/'q' row
    idle_time = group.loc[first_input_q_index, 'down_time']

    # Add it to the 'idle_time' column
    group['idle_time'] = idle_time

    # Drop all rows before this row
    group = group.loc[first_input_q_index:]

    # Subtract the 'down_time' of the first row from all 'down_time' values in the group
    group['up_time'] -= group['down_time'].iloc[0]
    group['down_time'] -= group['down_time'].iloc[0]

    return group

# Apply the function to each group
df = df.groupby('id').apply(process_group)

# Reset the index
df.reset_index(drop=True, inplace=True)

In [5]:
first_10min = 10*60*1000
first_20min = 20*60*1000

df_first = df[df['up_time'] < first_10min]

In [12]:
df_first.groupby('id')['up_time'].count().sort_values(ascending=False).tail(20)

id
5c88a769    41
bfc85aa5    17
ccbde4a9     7
3a266257     3
4b40b5c0     3
08eb331a     3
52bec922     2
2717fdef     2
f413c831     2
29b7f2f6     2
219e5338     2
fd565688     2
bf1a04b1     1
3e10785d     1
01963e20     1
31d5a61d     1
11612565     1
fb6c9c09     1
67d04f17     1
e6693dd9     1
Name: up_time, dtype: int64

In [8]:
df_first.to_csv('../kaggle feat_eng/train_logs_corrected_first_10min.csv', index=False)

In [9]:
df_second = df[(df['up_time'] >= first_10min) & (df['up_time'] < first_20min)]
df_third = df[df['up_time'] >= first_20min]

df_second.to_csv('../kaggle feat_eng/train_logs_corrected_second_10min.csv', index=False)
df_third.to_csv('../kaggle feat_eng/train_logs_corrected_third_10min.csv', index=False)

In [4]:
df[df['id']=='fff05981']

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
8402279,fff05981,1,39727,39880,153,Nonproduction,Leftclick,Leftclick,NoChange,0,0
8402280,fff05981,2,56401,56487,86,Nonproduction,Leftclick,Leftclick,NoChange,0,0
8402281,fff05981,3,76584,76719,135,Input,q,q,q,1,1
8402282,fff05981,4,76750,76862,112,Input,q,q,q,2,1
8402283,fff05981,5,76854,76959,105,Input,Space,Space,,3,1
...,...,...,...,...,...,...,...,...,...,...,...
8405893,fff05981,3615,2063944,2064440,496,Nonproduction,Leftclick,Leftclick,NoChange,1031,240
8405894,fff05981,3616,2064497,2064497,0,Nonproduction,Shift,Shift,NoChange,1031,240
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240


In [22]:
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified', 'Clear', 'PageUp', 'PageDown']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']

In [27]:
def get_avg_down_event_per_minute(df, events):

    # Convert max down_time from milliseconds to minutes for each id
    max_down_time = df.groupby('id')['down_time'].max() / (1000 * 60)

    # Count number of each unique down_event for each id
    event_count = df[df['down_event'].isin(events)].groupby(['id', 'down_event']).size().reset_index(name='counts')

    # Calculate average number of each unique down_event per minute
    event_count['avg_event_per_minute'] = event_count['counts'] / event_count['id'].map(max_down_time)

    # Pivot the DataFrame
    pivot_df = event_count.pivot(index='id', columns='down_event', values='avg_event_per_minute')

    # Fill NaN values with 0
    pivot_df = pivot_df.fillna(0)

    return pivot_df

In [28]:
get_avg_down_event_per_minute(df, events)

down_event,',",",.,ArrowDown,ArrowLeft,ArrowRight,ArrowUp,Backspace,CapsLock,Clear,Delete,Enter,Leftclick,PageDown,PageUp,Shift,Space,Unidentified,q
id,,,,,,,,,,,,,,,,,,,
001519c8,0.099896,0.399583,0.699271,0.000000,0.066597,0.066597,0.000000,13.885520,0.000000,0.0,0.0,0.133194,3.063472,0.0,0.0,0.899062,11.887604,0.0,53.910450
0022f953,0.100624,0.704366,0.503119,0.100624,1.643521,1.542898,0.067083,8.720725,0.000000,0.0,0.0,0.201248,1.878310,0.0,0.0,3.253501,13.114629,0.0,49.976465
0042269b,0.000000,0.779124,0.711374,0.000000,0.000000,0.203250,0.000000,14.871114,0.000000,0.0,0.0,0.575875,4.369872,0.0,0.0,1.321124,18.698986,0.0,98.372928
0059420b,0.085446,0.128169,0.555400,0.000000,0.000000,0.000000,0.000000,6.493904,0.085446,0.0,0.0,0.128169,0.769015,0.0,0.0,2.905168,10.381702,0.0,44.346530
0075873a,0.613574,0.866223,0.830130,0.000000,0.000000,0.000000,0.000000,18.659881,0.000000,0.0,0.0,0.360926,1.191056,0.0,0.0,1.407612,11.694007,0.0,55.618718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffb8c745,0.234430,1.071679,1.440069,0.000000,0.000000,0.000000,0.000000,32.150374,0.000000,0.0,0.0,0.234430,0.803759,0.0,0.0,5.492356,21.801973,0.0,95.245484
ffbef7e5,0.266797,0.800390,1.033836,0.000000,0.000000,0.000000,0.000000,2.000974,0.000000,0.0,0.0,0.400195,1.200584,0.0,0.0,3.535054,14.907255,0.0,62.497082
ffccd6fd,0.000000,0.061247,0.153118,0.888085,1.623051,0.888085,0.000000,2.694877,0.183742,0.0,0.0,0.367483,0.275612,0.0,0.0,0.000000,56.990527,0.0,29.674272


In [38]:


# Define a function to process the DataFrame
def calculate_deletions_per_min(df):

    def calc(df, event_name):
        # Convert down_time from milliseconds to seconds
        df['down_time_sec'] = df['down_time'] / 1000
        # Convert down_time_sec to datetime
        df['down_time_sec'] = pd.to_datetime(df['down_time_sec'], unit='s')
        # Set down_time_sec as index
        df.set_index('down_time_sec', inplace=True)
        # Calculate the length of deletions
        df['deletion_length'] = df['text_change'].str.len()
        # Resample to 1 minute intervals and sum deletion length
        df_sum = df.groupby('id').resample('1T')['deletion_length'].sum()
        # Calculate the average of the sums for each ID
        df_avg = df_sum.groupby('id').mean()
        # Reset index
        df_avg = df_avg.reset_index()
        # Rename the column
        df_avg.rename(columns={'deletion_length': f'{event_name}'}, inplace=True)

        return df_avg

    # Filter the DataFrame for 'Delete' down_event and 'q' text_change
    df_delete = df[(df['down_event']=='Delete')&(df['text_change']=='q')]
    
    # Repeat the same steps for 'Backspace'
    df_backspace = df[(df['down_event']=='Backspace')&(df['text_change']=='q')]

    temp1 = calc(df_delete, 'Delete')
    temp2 = calc(df_backspace, 'Backspace')

    # Merge the DataFrames
    df_avg = temp1.merge(temp2, on='id')

    return df_avg

# Process the DataFrames
df_del = calculate_deletions_per_min(df)

print(df_del)

C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\2120278322.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['down_time_sec'] = df['down_time'] / 1000
C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\2120278322.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['down_time_sec'] = pd.to_datetime(df['down_time_sec'], unit='s')
C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\2120278322.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

           id    Delete  Backspace
0    00f0737e  0.172414   6.689655
1    01963e20  7.900000  38.950000
2    02a41d1a  5.352941   9.846154
3    02cf6a52  2.200000   5.909091
4    031c0c58  3.000000  10.173913
..        ...       ...        ...
379  ff387a10  1.500000  10.400000
380  ff5ac707  0.800000   2.310345
381  ff635a91  1.000000   7.583333
382  ff7f1779  1.000000  12.833333
383  ff9a11ae  1.952381  19.448276

[384 rows x 3 columns]


In [43]:
def calculate_insertions_per_min(df):

    df_q = df[(df['down_event']=='q')&(df['text_change']=='q')]

    # Convert down_time from milliseconds to seconds
    df_q['down_time_sec'] = df_q['down_time'] / 1000
    # Convert down_time_sec to datetime
    df_q['down_time_sec'] = pd.to_datetime(df_q['down_time_sec'], unit='s')
    # Set down_time_sec as index
    df_q.set_index('down_time_sec', inplace=True)
    # Calculate the length of deletions
    df_q['insert_q'] = df_q['text_change'].str.len()
    # Resample to 1 minute intervals and sum deletion length
    df_sum = df_q.groupby('id').resample('1T')['insert_q'].sum()
    # Calculate the average of the sums for each ID
    df_avg = df_sum.groupby('id').mean()
    # Reset index}
    df_avg = df_avg.reset_index()

    return df_avg

# Process the DataFrames
df_q = calculate_insertions_per_min(df)

df_q

C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\3697091739.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q['down_time_sec'] = df_q['down_time'] / 1000
C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\3697091739.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q['down_time_sec'] = pd.to_datetime(df_q['down_time_sec'], unit='s')
C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\3697091739.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,id,insert_q
0,001519c8,55.620690
1,0022f953,55.148148
2,0042269b,96.566667
3,0059420b,47.181818
4,0075873a,59.269231
...,...,...
2466,ffb8c745,105.296296
2467,ffbef7e5,62.433333
2468,ffccd6fd,38.760000
2469,ffec5b38,102.652174
